<a href="https://colab.research.google.com/github/alsantosad/PROJECT-ROCKET/blob/backstage/M4TRABALHO_EM_GRUPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**PROJETO EM GRUPO - MÓDULO 4**

## ***CONTEXTO***
Você e a sua equipe foram escalados pela Agência Nacional de Petróleo e Gás
Natural e Biocombustíveis (ANP) para realizar uma análise exploratória
relacionada à série histórica de preço de venda da gasolina e do etanol.

A ANP é um centro de referência em dados e informações sobre a indústria de petróleo e gás natural no Brasil, que mantém o Banco de Dados de Exploração e
Produção (BDEP); divulga as estatísticas oficiais sobre reservas, refino e produção
de petróleo, gás e biocombustíveis no Brasil;
realiza e divulga pesquisas sobre
qualidade dos combustíveis e lubrificantes e sobre o comportamento dos preços
desses produtos; promove estudos sobre o potencial petrolífero e o
desenvolvimento do setor; entre outras atividades

Realizar a análise exploratória a partir das fontes de dados
disponíveis no portal gov.br.

Serão utilizados os arquivos
dos dois últimos meses do ano atual (Etanol Hidratado +
Gasolina C) contendo a série histórica dos preços da
gasolina e do etanol em todo o Brasil.


A análise deve responder às seguintes perguntas:
1. Como se comportaram o preço dos combustíveis durante os dois meses citados? Os valores do
etanol e da gasolina tiveram uma tendência de queda ou aumento?
2. Qual o preço médio da gasolina e do etanol nesses dois meses?
3. Quais os 5 estados com o preço médio da gasolina e do etanol mais caros?
4. Qual o preço médio da gasolina e do etanol por estado?
5. Qual o município que possui o menor preço para a gasolina e para o etanol?
6. Qual o município que possui o maior preço para a gasolina e para o etanol?
7. Qual a região que possui o maior valor médio da gasolina?
8. Qual a região que possui o menor valor médio do etanol?
9. Há alguma correlação entre o valor do combustível (gasolina e etanol) e a região onde ele é vendido?
10. Há alguma correlação entre o valor do combustível (gasolina e etanol) e a bandeira que vende ele?
Além disso, vocês deverão pensar em mais três perguntas que deverão ser respondidas por scripts.

Questões extras:

11. Qual a distribuição de bandeiras por região do país?
12. Existe alguma relação entre a data do mês em que determinadas bandeiras são mais lucrativas?
13. Existe alguma bandeira exclusiva de alguns estados?

## ***Importando Bibliotecas***

In [73]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
import unidecode as uni

## ***Analise Exploratória***

### *Leitura dos Datasets*

In [74]:
#Lendo os csv pelo link do site

# Primeiro arquivo referente ao mês 11
url1 = 'https://raw.githubusercontent.com/alsantosad/PROJECT-ROCKET/master/DADOS/precos-gasolina-etanol-11.csv'
df1 = pd.read_csv(url1,sep=';')

# Segundo arquivo referente ao mês 12
url2 = 'https://raw.githubusercontent.com/alsantosad/PROJECT-ROCKET/master/DADOS/precos-gasolina-etanol-12.csv'
df2 = pd.read_csv(url2,sep=';')

# Leitura do dataset extra de latitude e longitude de cidades para geolocalização
municipios ='https://raw.githubusercontent.com/kelvins/municipios-brasileiros/main/csv/municipios.csv'
municipios_df = pd.read_csv(municipios,sep=',')

### *Visualizando os Datasets Separados*

In [75]:
#Visualizando os dados do data set
df1.head(2)

,Regiao - Sigla,Estado - Sigla,Municipio,Revenda,CNPJ da Revenda,Nome da Rua,Numero Rua,Complemento,Bairro,Cep,Produto,Data da Coleta,Valor de Venda,Valor de Compra,Unidade de Medida,Bandeira
0,NE,AL,PALMEIRA DOS INDIOS,SEVERINO S. LOPES & CIA. LTDA,05.331.412/0002-09,AVENIDA MUNIZ FALCÃO,1640,"LOTES 08 E 09, LOT. GASTÃO LEÃO REGO",JOSÉ MAIA COSTA,57605-260,GASOLINA,01/11/2023,"6,3",NaN,R$ / litro,VIBRA
1,NE,AL,PALMEIRA DOS INDIOS,SEVERINO S. LOPES & CIA. LTDA,05.331.412/0002-09,AVENIDA MUNIZ FALCÃO,1640,"LOTES 08 E 09, LOT. GASTÃO LEÃO REGO",JOSÉ MAIA COSTA,57605-260,GASOLINA ADITIVADA,01/11/2023,"6,3",NaN,R$ / litro,VIBRA


In [76]:
#Visualizando os dados do data set
df2.head(2)

,Regiao - Sigla,Estado - Sigla,Municipio,Revenda,CNPJ da Revenda,Nome da Rua,Numero Rua,Complemento,Bairro,Cep,Produto,Data da Coleta,Valor de Venda,Valor de Compra,Unidade de Medida,Bandeira
0,NE,MA,SANTA INES,POSTO MAGNOLIA LTDA,35.123.447/0001-32,AVENIDA MARECHAL CASTELO BRANCO,3707,NaN,CENTRO,65300-001,GASOLINA,01/12/2023,"5,59",NaN,R$ / litro,BRANCA
1,NE,MA,SANTA INES,POSTO MAGNOLIA LTDA,35.123.447/0001-32,AVENIDA MARECHAL CASTELO BRANCO,3707,NaN,CENTRO,65300-001,GASOLINA ADITIVADA,01/12/2023,"5,74",NaN,R$ / litro,BRANCA


In [77]:
#Visualizando numero de linhas e colunas dos 2 Datasets
print(df1.shape,df2.shape)

(52543, 16) (49453, 16)


###*Juntando os Datasets com o método `concat()`*

In [78]:
#Datasets juntos `df1, df2` e ignorando o index para melhor organização do Dataset
#Ignore index ajuda não dar erro
df = pd.concat([df1, df2],ignore_index=True)
df.head(2)

,Regiao - Sigla,Estado - Sigla,Municipio,Revenda,CNPJ da Revenda,Nome da Rua,Numero Rua,Complemento,Bairro,Cep,Produto,Data da Coleta,Valor de Venda,Valor de Compra,Unidade de Medida,Bandeira
0,NE,AL,PALMEIRA DOS INDIOS,SEVERINO S. LOPES & CIA. LTDA,05.331.412/0002-09,AVENIDA MUNIZ FALCÃO,1640,"LOTES 08 E 09, LOT. GASTÃO LEÃO REGO",JOSÉ MAIA COSTA,57605-260,GASOLINA,01/11/2023,"6,3",NaN,R$ / litro,VIBRA
1,NE,AL,PALMEIRA DOS INDIOS,SEVERINO S. LOPES & CIA. LTDA,05.331.412/0002-09,AVENIDA MUNIZ FALCÃO,1640,"LOTES 08 E 09, LOT. GASTÃO LEÃO REGO",JOSÉ MAIA COSTA,57605-260,GASOLINA ADITIVADA,01/11/2023,"6,3",NaN,R$ / litro,VIBRA


## ***Limpeza dos Dados que vamos Trabalhar***

###*Visualizando as valores da coluna de varias formas*

In [79]:
#Visualizando as informações da coluna "Valor de Compra"
df['Valor de Compra'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 101996 entries, 0 to 101995
Series name: Valor de Compra
Non-Null Count  Dtype  
--------------  -----  
0 non-null      float64
dtypes: float64(1)
memory usage: 797.0 KB


In [80]:
# Visualizando a descrição
df['Valor de Compra'].describe

<bound method NDFrame.describe of 0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
          ..
101991   NaN
101992   NaN
101993   NaN
101994   NaN
101995   NaN
Name: Valor de Compra, Length: 101996, dtype: float64>

In [81]:
# Visualizando os tipos de valores
df['Valor de Compra'].value_counts

<bound method IndexOpsMixin.value_counts of 0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
          ..
101991   NaN
101992   NaN
101993   NaN
101994   NaN
101995   NaN
Name: Valor de Compra, Length: 101996, dtype: float64>

In [82]:
#Visualizando os Tipos unicos de Valores na coluna
#Interessante, a função (.unique()) retorna um array
#A função (.unique) sem parentese retorna uma serie
#Diferentes metodos de visualização
df['Valor de Compra'].unique()

array([nan])

###*Corrigindo valores das colunas*

In [83]:
# Corrigindo valores em "Valor de Venda". Substituindo as (,)virgulas por (.)ponto.
df['Valor de Venda'] = df['Valor de Venda'].str.replace(',', '.').astype(float)

In [84]:
# Formatando a coluna Data da Coleta de str para DATE utilizando a função TO_DATETIME
df['Data da Coleta'] = pd.to_datetime(df['Data da Coleta'], format='%d/%m/%Y')
df

,Regiao - Sigla,Estado - Sigla,Municipio,Revenda,CNPJ da Revenda,Nome da Rua,Numero Rua,Complemento,Bairro,Cep,Produto,Data da Coleta,Valor de Venda,Valor de Compra,Unidade de Medida,Bandeira
0,NE,AL,PALMEIRA DOS INDIOS,SEVERINO S. LOPES & CIA. LTDA,05.331.412/0002-09,AVENIDA MUNIZ FALCÃO,1640,"LOTES 08 E 09, LOT. GASTÃO LEÃO REGO",JOSÉ MAIA COSTA,57605-260,GASOLINA,2023-11-01,6.30,NaN,R$ / litro,VIBRA
1,NE,AL,PALMEIRA DOS INDIOS,SEVERINO S. LOPES & CIA. LTDA,05.331.412/0002-09,AVENIDA MUNIZ FALCÃO,1640,"LOTES 08 E 09, LOT. GASTÃO LEÃO REGO",JOSÉ MAIA COSTA,57605-260,GASOLINA ADITIVADA,2023-11-01,6.30,NaN,R$ / litro,VIBRA
2,NE,AL,PALMEIRA DOS INDIOS,SEVERINO S. LOPES & CIA. LTDA,05.331.412/0002-09,AVENIDA MUNIZ FALCÃO,1640,"LOTES 08 E 09, LOT. GASTÃO LEÃO REGO",JOSÉ MAIA COSTA,57605-260,ETANOL,2023-11-01,4.88,NaN,R$ / litro,VIBRA
3,NE,AL,PALMEIRA DOS INDIOS,A. B. LESSA NETO & CIA LTDA,05.562.589/0001-35,AVENIDA VIEIRA DE BRITO,1888,NaN,VILA MARIA,57607-570,GASOLINA,2023-11-01,6.19,NaN,R$ / litro,VIBRA
4,NE,AL,PALMEIRA DOS INDIOS,A. B. LESSA NETO & CIA LTDA,05.562.589/0001-35,AVENIDA VIEIRA DE BRITO,1888,NaN,VILA MARIA,57607-570,GASOLINA ADITIVADA,2023-11-01,6.29,NaN,R$ / litro,VIBRA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101991,S,PR,CAMPO LARGO,AUTO POSTO IMPERADOR DE CAMPO LARGO LTDA,42.182.809/0001-21,RUA XAVIER DA SILVA,1596,NaN,CENTRO,83601-010,GASOLINA,2023-12-29,5.79,NaN,R$ / litro,IPIRANGA
101992,S,PR,CAMPO LARGO,AUTO POSTO IMPERADOR DE CAMPO LARGO LTDA,42.182.809/0001-21,RUA XAVIER DA SILVA,1596,NaN,CENTRO,83601-010,ETANOL,2023-12-29,3.79,NaN,R$ / litro,IPIRANGA
101993,S,RS,OSORIO,POSTO ALDEIAO COMERCIO DE COMBUSTIVEIS LTDA,28.259.725/0002-07,AVENIDA JORGE DARIVA,1536,NaN,CENTRO,95520-000,GASOLINA,2023-12-29,5.49,NaN,R$ / litro,RODOIL
101994,S,RS,OSORIO,POSTO ALDEIAO COMERCIO DE COMBUSTIVEIS LTDA,28.259.725/0002-07,AVENIDA JORGE DARIVA,1536,NaN,CENTRO,95520-000,GASOLINA ADITIVADA,2023-12-29,5.59,NaN,R$ / litro,RODOIL


###*Limpeza dos Dados, Removendo colunas indesejadas*

In [85]:
#Removendo colunas que não serão uteis.
dfproc = df.drop(columns=['Valor de Compra','Nome da Rua','Numero Rua','Complemento','Cep','CNPJ da Revenda','Revenda', 'Unidade de Medida'])
dfproc.head(2)

,Regiao - Sigla,Estado - Sigla,Municipio,Bairro,Produto,Data da Coleta,Valor de Venda,Bandeira
0,NE,AL,PALMEIRA DOS INDIOS,JOSÉ MAIA COSTA,GASOLINA,2023-11-01,6.3,VIBRA
1,NE,AL,PALMEIRA DOS INDIOS,JOSÉ MAIA COSTA,GASOLINA ADITIVADA,2023-11-01,6.3,VIBRA


In [86]:
#Visualizando as informações da coluna, pois vamos trabalhar com ela.
dfproc['Valor de Venda'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 101996 entries, 0 to 101995
Series name: Valor de Venda
Non-Null Count   Dtype  
--------------   -----  
101996 non-null  float64
dtypes: float64(1)
memory usage: 797.0 KB


In [87]:
#Removendo colunas que não serão uteis também no dataset adicional
municipios_df = municipios_df.drop(columns=['codigo_uf','codigo_ibge','capital', 'siafi_id', 'ddd', 'fuso_horario'])
municipios_df.head(2)

,nome,latitude,longitude
0,Abadia de Goiás,-16.7573,-49.4412
1,Abadia dos Dourados,-18.4831,-47.3916


##***Organizando***

###*Separando em Regiões*

In [88]:
#Ordenando as regiões
regioes = dfproc.sort_values('Regiao - Sigla')
regioes = pd.DataFrame(regioes)

#Verificando a reordenação
regioes.head(5)

,Regiao - Sigla,Estado - Sigla,Municipio,Bairro,Produto,Data da Coleta,Valor de Venda,Bandeira
88438,CO,MS,PONTA PORA,VILA AQUIDABAM,GASOLINA,2023-12-21,5.74,IPIRANGA
6372,CO,DF,BRASILIA,RIACHO FUNDO,GASOLINA ADITIVADA,2023-11-06,5.78,VIBRA
6371,CO,DF,BRASILIA,RIACHO FUNDO,GASOLINA,2023-11-06,5.78,VIBRA
6370,CO,DF,BRASILIA,SAMAMBAIA,ETANOL,2023-11-06,3.73,VIBRA
6369,CO,DF,BRASILIA,SAMAMBAIA,GASOLINA ADITIVADA,2023-11-06,5.79,VIBRA


### *Utilizando método `.merge` com Dataset extra para geolocalização*

Primeiro precisamos verificar se os Dtype são iguais para que seja possível fazer o merge.

Se não for, fazemos a alteração usando o método `.astype()`

In [89]:
#Verificando as informações dos DataSets
dfproc.info()
municipios_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101996 entries, 0 to 101995
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   Regiao - Sigla  101996 non-null  object        
 1   Estado - Sigla  101996 non-null  object        
 2   Municipio       101996 non-null  object        
 3   Bairro          101810 non-null  object        
 4   Produto         101996 non-null  object        
 5   Data da Coleta  101996 non-null  datetime64[ns]
 6   Valor de Venda  101996 non-null  float64       
 7   Bandeira        101996 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(6)
memory usage: 6.2+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5570 entries, 0 to 5569
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   nome       5570 non-null   object 
 1   latitude   5570 non-null   float64
 2   longitude  5570 no

### *Normalizando os dados antes da mesclagem*

In [90]:
#Criando uma função para tirar os acentos
def tira_acentos(nome):
    txt = uni.unidecode(str(nome))
    return txt

In [91]:
#Colocando tudo em Caps Lock
municipios_df['nome'] = municipios_df['nome'].str.upper()

# Nova DF sem acentos = df antiga com acentos + método .apply()
municipios_df['nomes_cidades'] = municipios_df['nome'].apply(tira_acentos)

#Removendo coluna desnecessária

municipios_df = municipios_df.drop(columns=['nome'])
municipios_df.head(2)


,latitude,longitude,nomes_cidades
0,-16.7573,-49.4412,ABADIA DE GOIAS
1,-18.4831,-47.3916,ABADIA DOS DOURADOS


###*Mesclando*

In [92]:
#Selecionamos os dataframes que queremos mesclar, definimos o método e quais são a cardinalidade
df_geoloc = pd.merge(regioes, municipios_df, how= 'left', left_on='Municipio', right_on='nomes_cidades')
df_geoloc.head(2)

,Regiao - Sigla,Estado - Sigla,Municipio,Bairro,Produto,Data da Coleta,Valor de Venda,Bandeira,latitude,longitude,nomes_cidades
0,CO,MS,PONTA PORA,VILA AQUIDABAM,GASOLINA,2023-12-21,5.74,IPIRANGA,-22.5296,-55.7203,PONTA PORA
1,CO,DF,BRASILIA,RIACHO FUNDO,GASOLINA ADITIVADA,2023-11-06,5.78,VIBRA,-15.7795,-47.9297,BRASILIA


In [93]:
#Removemos novamente a coluna desnecessária
df_geoloc = df_geoloc.drop(columns=['nomes_cidades'])
df_geoloc.head(2)

,Regiao - Sigla,Estado - Sigla,Municipio,Bairro,Produto,Data da Coleta,Valor de Venda,Bandeira,latitude,longitude
0,CO,MS,PONTA PORA,VILA AQUIDABAM,GASOLINA,2023-12-21,5.74,IPIRANGA,-22.5296,-55.7203
1,CO,DF,BRASILIA,RIACHO FUNDO,GASOLINA ADITIVADA,2023-11-06,5.78,VIBRA,-15.7795,-47.9297


In [94]:
df_geoloc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 110255 entries, 0 to 110254
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   Regiao - Sigla  110255 non-null  object        
 1   Estado - Sigla  110255 non-null  object        
 2   Municipio       110255 non-null  object        
 3   Bairro          110043 non-null  object        
 4   Produto         110255 non-null  object        
 5   Data da Coleta  110255 non-null  datetime64[ns]
 6   Valor de Venda  110255 non-null  float64       
 7   Bandeira        110255 non-null  object        
 8   latitude        110073 non-null  float64       
 9   longitude       110073 non-null  float64       
dtypes: datetime64[ns](1), float64(3), object(6)
memory usage: 9.3+ MB


In [95]:
df_geoloc[''].unique()

KeyError: 'Unidade de Medida'

##***Perguntas***

###*1. Como se comportaram o preço dos combustíveis durante os dois meses citados? Os valores do etanol e da gasolina tiveram uma tendência de queda ou diminuição?*

In [ ]:
#Verificando se o média do Valor do Etanol está de acordo com o gráfico mostrado
dfproc[dfproc['Produto'] == 'ETANOL']['Valor de Venda'].mean

In [ ]:
#Achando o Minimo e o Maximo do Valor de Venda do ETANOL
maior_valor_etanol = dfproc[dfproc['Produto'] == 'ETANOL']['Valor de Venda'].max()
menor_valor_etanol = dfproc[dfproc['Produto'] == 'ETANOL']['Valor de Venda'].min()
display(f'O maior valor do Etanol é {maior_valor_etanol}')
display(f'O menor valor do Etanol é {menor_valor_etanol}')

In [ ]:
#Buscando o maior valor de Venda do Etanol (Verificando Outlier)
maior_valor_etanol = dfproc[(dfproc['Produto'] == 'ETANOL') & (dfproc['Valor de Venda'] == 6.6)]
maior_valor_etanol.head(4)

In [ ]:
# Supondo que 'dados' é o DataFrame que contém seus dados
media_etanol = dfproc[dfproc['Produto'] == 'ETANOL']['Valor de Venda'].mean()

print(f'Média do valor de venda do Etanol: {media_etanol}')


In [ ]:
# Agrupando por semana e tipo de combustível, e calculando a média dos preços de venda
comport = dfproc.groupby([pd.Grouper(key='Data da Coleta', freq='W'), 'Produto'])['Valor de Venda'].median().reset_index()
#Plotando o gráfico de linha
fig, ax = plt.subplots(figsize=(10, 6))
plt.xticks(rotation=45)
sns.lineplot(data=comport, x='Data da Coleta', y='Valor de Venda', hue='Produto', ax=ax)
plt.show()

In [ ]:
comport = dfproc.groupby([pd.Grouper(key='Data da Coleta', freq='W'), 'Produto'])['Valor de Venda'].median()
comport

In [ ]:
dfproc.head(2)

###*2. Qual o preço médio da gasolina e do etanol nesses dois meses?*

In [ ]:
# Filtrar apenas as linhas relacionadas à gasolina
gasolina_df = dfproc[dfproc['Produto'] == 'GASOLINA']

# Filtrar apenas as linhas relacionadas ao etanol
etanol_df = dfproc[dfproc['Produto'] == 'ETANOL']

# Calcular o preço médio da gasolina
preco_medio_gasolina = gasolina_df['Valor de Venda'].mean()

# Calcular o preço médio do etanol
preco_medio_etanol = etanol_df['Valor de Venda'].mean()

# Dados para o gráfico
produtos = ['Gasolina', 'Etanol']
precos_medios = [preco_medio_gasolina, preco_medio_etanol]

# Criar o gráfico de barras
plt.bar(produtos, precos_medios, color=['orange', 'blue'])

# Adicionar rótulos e título ao gráfico
plt.xlabel('Produto')
plt.ylabel('Preço Médio (R$/litro)')
plt.title('Preço Médio da Gasolina e Etanol')

# Mostrar o gráfico
plt.show()

###*3. Quais os 5 estados com o preço médio da gasolina e do etanol mais caros?*

In [ ]:
dfproc.head(3)

In [ ]:
# Filtra apenas as linhas relacionadas à gasolina
gasolina_df = dfproc[dfproc['Produto'] == 'GASOLINA']

# Filtra apenas as linhas relacionadas ao etanol
etanol_df = dfproc[dfproc['Produto'] == 'ETANOL']

# Calcula o preço médio da gasolina para cada estado
media_gasolina_por_estado = gasolina_df.groupby('Estado - Sigla')['Valor de Venda'].mean()

# Calcula o preço médio do etanol para cada estado
media_etanol_por_estado = etanol_df.groupby('Estado - Sigla')['Valor de Venda'].mean()

# Obtém os 5 estados com os preços médios mais altos para a gasolina
top5_gasolina = media_gasolina_por_estado.nlargest(5)

# Obtém os 5 estados com os preços médios mais altos para o etanol
top5_etanol = media_etanol_por_estado.nlargest(5)

# Exibe os resultados
print('Top 5 Estados com Preço Médio mais Alto para Gasolina:')
print(top5_gasolina)

print('\nTop 5 Estados com Preço Médio mais Alto para Etanol:')
print(top5_etanol)


In [ ]:
# Configuração do gráfico de barras para a gasolina
plt.figure(figsize=(10, 5))
plt.barh(top5_gasolina.index, top5_gasolina, color='orange', label='Gasolina')

# Configuração do gráfico de barras para o etanol
plt.barh(top5_etanol.index, top5_etanol, color='blue', label='Etanol')

# Adiciona rótulos e título ao gráfico
plt.xlabel('Preço Médio (R$/litro)')
plt.ylabel('Estado')
plt.title('Top 5 Estados com Preço Médio mais Alto para Gasolina e Etanol')
plt.legend()

# Mostra o gráfico
plt.show()


###*4. Qual o preço médio da gasolina e do etanol por estado?*

In [ ]:
# Filtra apenas as linhas relacionadas à gasolina
gasolina_df = dfproc[dfproc['Produto'] == 'GASOLINA']

# Filtra apenas as linhas relacionadas ao etanol
etanol_df = dfproc[dfproc['Produto'] == 'ETANOL']

# Calcula o preço médio da gasolina para cada estado
media_gasolina_por_estado = gasolina_df.groupby('Estado - Sigla')['Valor de Venda'].mean()

# Calcula o preço médio do etanol para cada estado
media_etanol_por_estado = etanol_df.groupby('Estado - Sigla')['Valor de Venda'].mean()

print(f'A média do Preço da Gasolina por Estado é {media_gasolina_por_estado}')
print(f'A média do Preço do Etanol por Estado é {media_etanol_por_estado}')

In [ ]:
# Juntando os dataframes para facilitar a visualização
df = pd.DataFrame({'Gasolina': media_gasolina_por_estado, 'Etanol': media_etanol_por_estado}).reset_index()

# Criando o gráfico de linhas usando seaborn com cores personalizadas
plt.figure(figsize=(10, 6))
sns.lineplot(data=df.melt('Estado - Sigla', var_name='Combustível', value_name='Preço Médio (R$/litro)'),
             x='Estado - Sigla', y='Preço Médio (R$/litro)', hue='Combustível', marker='o', palette={'Gasolina': 'orange', 'Etanol': 'blue'})
plt.title('Preço Médio da Gasolina e Etanol por Estado (com linha)')
plt.xlabel('Estado - Sigla')
plt.ylabel('Preço Médio (R$/litro)')

plt.show()

###*5. Qual o município que possui o menor preço para a gasolina e para o etanol?*

In [ ]:
# Calcula o preço minimo da gasolina para cada estado
min_gasolina_por_municipio = gasolina_df.groupby('Municipio')['Valor de Venda'].min()
municipio_menor_gasolina = min_gasolina_por_municipio.sort_values().head(1)
df_municipio_menor_gasolina = pd.DataFrame(municipio_menor_gasolina)
df_municipio_menor_gasolina

In [ ]:
# Calcula o preço minimo do etanol para cada municipio
min_etanol_por_municipio = etanol_df.groupby('Municipio')['Valor de Venda'].min()
municipio_menor_etanol = min_etanol_por_municipio.sort_values().head(1)
df_municipio_menor_etanol = pd.DataFrame(municipio_menor_etanol)
df_municipio_menor_etanol

###*6. Qual o município que possui o maior preço para a gasolina e para o etanol?*


In [ ]:
# Calcula o preço maximo da gasolina para cada municipio
max_gasolina_por_municipio = gasolina_df.groupby('Municipio')['Valor de Venda'].max()
municipio_max_gasolina = max_gasolina_por_municipio.sort_values().head(1)
df_municipio_max_gasolina = pd.DataFrame(municipio_max_gasolina)
df_municipio_max_gasolina

In [ ]:
# Calcula o preço maximo do etanol para cada estado
max_etanol_por_municipio = etanol_df.groupby('Municipio')['Valor de Venda'].max()
municipio_max_etanol = max_etanol_por_municipio.sort_values().head(1)
df_municipio_max_etanol=pd.DataFrame(municipio_max_etanol)
df_municipio_max_etanol

###*7. Qual a região que possui o maior valor médio da gasolina?*

In [ ]:
media_gasolina_por_regiao = gasolina_df.groupby("Regiao - Sigla")['Valor de Venda'].mean()

regiao_maior_media_gasolina = media_gasolina_por_regiao.idxmax()
valor_maior_media_gasolina = media_gasolina_por_regiao.max()

print(f'A região com o maior valor médio de gasolina é {regiao_maior_media_gasolina} com média de {valor_maior_media_gasolina:.2f} R$/litro.')

regioes_ordenadas = media_gasolina_por_regiao.sort_values(ascending=False)

df_regiao_max_gasolina = pd.DataFrame(regioes_ordenadas)
df_regiao_max_gasolina

###*8.Qual a região que possui o menor valor médio do etanol?*

In [ ]:
media_etanol_por_regiao = etanol_df.groupby("Regiao - Sigla")['Valor de Venda'].mean()

regiao_maior_media_etanol = media_etanol_por_regiao.idxmax()
valor_maior_media_etanol = media_etanol_por_regiao.max()

print(f'A região com o maior valor médio de Etanol é {regiao_maior_media_etanol} com média de {valor_maior_media_etanol:.2f} R$/litro.')

regioes_ordenadas = media_etanol_por_regiao.sort_values(ascending=False)

df_regiao_media_etanol = pd.DataFrame(regioes_ordenadas)
df_regiao_media_etanol

###*9. Há alguma correlação entre o valor do combustível (gasolina e etanol) e a região onde ele é vendido?*

#### ***Correlação do Etanol e a Região***

In [ ]:
# Selecionando o etanol da região norte
norte_etanol = dfproc.loc[(dfproc['Regiao - Sigla'] == 'N') & (dfproc['Produto'] == 'ETANOL'), 'Valor de Venda'].astype(float)

# Selecionando o etanol da região nordeste
nordeste_etanol = dfproc.loc[(dfproc['Regiao - Sigla'] == 'NE') & (dfproc['Produto'] == 'ETANOL'), 'Valor de Venda'].astype(float)

# Selecionando o etanol da região Sul
sul_etanol = dfproc.loc[(dfproc['Regiao - Sigla'] == 'S') & (dfproc['Produto'] == 'ETANOL'), 'Valor de Venda'].astype(float)

# Selecionando o etanol da região Suldeste
suldeste_etanol = dfproc.loc[(dfproc['Regiao - Sigla'] == 'SE') & (dfproc['Produto'] == 'ETANOL'), 'Valor de Venda'].astype(float)

# Selecionando o etanol da região Centro Oeste
co_etanol = dfproc.loc[(dfproc['Regiao - Sigla'] == 'CO') & (dfproc['Produto'] == 'ETANOL'), 'Valor de Venda'].astype(float)

# Criando um Data frame com as informações filtradas
etanol_dados_regiao = pd.DataFrame({'Norte':norte_etanol,'Nordeste':nordeste_etanol,'Sul':sul_etanol,'Sudeste':suldeste_etanol,'Centro Oeste':co_etanol})

etanol_dados_regiao.fillna(0).corr()

In [ ]:
etanol_dados_regiao.isnull().shape[0]

In [ ]:
etanol_dados_regiao.shape

In [ ]:
etanol_dados_regiao

In [ ]:
# Preenchendo valores nulos com 0 e calculando a correlação
correlation_matrix_etanol = etanol_dados_regiao.fillna(0).corr()

# Criando o mapa de calor
plt.figure(figsize=(6, 4))
sns.heatmap(correlation_matrix_etanol, annot=True, cmap="coolwarm", fmt=".2f", linewidths=.5)
plt.title('Correlação do Valor de Venda de Gasolina por Região')
plt.show()

#### ***Correlação do Gasolina e a Região***

In [ ]:
# Selecionando o Gasolina da região norte
norte_gasolina = dfproc.loc[(dfproc['Regiao - Sigla'] == 'N') & (dfproc['Produto'] == 'GASOLINA'), 'Valor de Venda'].astype(float)

# Selecionando o Gasolina da região nordeste
nordeste_gasolina = dfproc.loc[(dfproc['Regiao - Sigla'] == 'NE') & (dfproc['Produto'] == 'GASOLINA'), 'Valor de Venda'].astype(float)

# Selecionando o Gasolina da região Sul
sul_gasolina = dfproc.loc[(dfproc['Regiao - Sigla'] == 'S') & (dfproc['Produto'] == 'GASOLINA'), 'Valor de Venda'].astype(float)

# Selecionando o Gasolina da região Suldeste
suldeste_gasolina = dfproc.loc[(dfproc['Regiao - Sigla'] == 'SE') & (dfproc['Produto'] == 'GASOLINA'), 'Valor de Venda'].astype(float)

# Selecionando o Gasolina da região Centro Oeste
co_gasolina = dfproc.loc[(dfproc['Regiao - Sigla'] == 'CO') & (dfproc['Produto'] == 'GASOLINA'), 'Valor de Venda'].astype(float)

# Criando um Data frame com as informações filtradas
gasolina_dados_regiao = pd.DataFrame({'Norte':norte_gasolina,'Nordeste':nordeste_gasolina,'Sul':sul_gasolina,'Sudeste':suldeste_gasolina,'Centro Oeste':co_gasolina})

gasolina_dados_regiao.fillna(0).corr()

In [ ]:
# Preenchendo valores nulos com 0 e calculando a correlação
correlation_matrix_gasol = gasolina_dados_regiao.fillna(0).corr()

# Criando o mapa de calor
plt.figure(figsize=(6, 4))
sns.heatmap(correlation_matrix_gasol, annot=True, cmap="coolwarm", fmt=".2f", linewidths=.5)
plt.title('Correlação do Valor de Venda de Gasolina por Região')
plt.show()

###*10. Há alguma correlação entre o valor do combustível (gasolina e etanol) e a bandeira que vende ele?*

#### *Visualizando e tratando DataFrame*

In [ ]:
# Filtrar apenas as linhas relacionadas a combustíveis
combustiveis_df = dfproc[dfproc['Produto'].isin(['GASOLINA', 'ETANOL'])]

# Agrupar por Bandeira e Produto, calculando a média do Valor de Venda
agrupado_combustiveis = combustiveis_df.groupby(['Bandeira','Regiao - Sigla','Data da Coleta','Produto'])['Valor de Venda'].mean()

In [ ]:
# Separando os dados dos Combustivel em colunas para facilitar o entendimento
combustivel_bandeira = agrupado_combustiveis.unstack()
combustivel_bandeira

In [ ]:
# Utilizamos essa função para dropar a coluna Gasolina, criar um novo dataframe e renomear o nome de
# index que estava como Produto, removendo o nome e deixando o index sem nome e reindexando.
etanol = combustivel_bandeira.drop('GASOLINA', axis=1).copy().rename_axis(None, axis=1).reset_index()
etanol

### ***Região Nordeste***

#### ***Etanol***

In [ ]:
# Selecionando a Região do Etanol
etanol_ne = etanol[etanol['Regiao - Sigla'] == 'NE']
etanol_ne

In [ ]:
# Criando um novo DataFrame com o Index:Data da Coleta, Colunas: Bandeira, Valores: Etanol para fazer uma correlação.
etanol_ne_novodf = etanol_ne.pivot(index='Data da Coleta', columns='Bandeira', values='ETANOL')
etanol_ne_novodf

In [ ]:
# Fazendo a correlação do preço do etanol de bandeira para outra bandeira.
# Filtrando os dados faltantes com o Fillna()
etanol_correlacao_ne = etanol_ne_novodf.fillna(0).corr()
etanol_correlacao_ne

In [ ]:
# Criando um heatmap para a correlação usando a função plt
plt.figure(figsize=(12, 10))
sns.heatmap(etanol_correlacao_ne, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
plt.title('Correlação entre Bandeiras de Etanol na Região Nordeste')
plt.show()

#### ***Gasolina***

In [ ]:
# Utilizamos essa função para dropar a coluna Etanol, criar um novo dataframe e renomear o nome de
# index que estava como Produto, removendo o nome e deixando o index sem nome e reindexando.
gasolina = combustivel_bandeira.drop('ETANOL', axis=1).copy().rename_axis(None, axis=1).reset_index()
# Selecionando a Região da Gasolina
gasolina_ne = gasolina[gasolina['Regiao - Sigla'] == 'NE']
# Criando um novo DataFrame com o Index:Data da Coleta, Colunas: Bandeira, Valores: Gasolina para fazer uma correlação.
gasolina_ne_novodf = gasolina_ne.pivot(index='Data da Coleta', columns='Bandeira', values='GASOLINA')
# Fazendo a correlação do preço da Gasolina de bandeira para outra bandeira.
# Filtrando os dados faltantes com o Fillna()
gasolina_correlacao_ne = gasolina_ne_novodf.fillna(0).corr()
# Criando um heatmap para a correlação usando a função plt
plt.figure(figsize=(12, 10))
sns.heatmap(gasolina_correlacao_ne, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
plt.title('Correlação entre Bandeiras da Gasolina na Região Nordeste')
plt.show()

### ***Região Norte***

#### ***Etanol***

In [ ]:
# Utilizamos essa função para dropar a coluna Gasolina, criar um novo dataframe e renomear o nome de
# index que estava como Produto, removendo o nome e deixando o index sem nome e reindexando.
etanol = combustivel_bandeira.drop('GASOLINA', axis=1).copy().rename_axis(None, axis=1).reset_index()
# Selecionando a Região do Etanol
etanol_n = etanol[etanol['Regiao - Sigla'] == 'N']
# Criando um novo DataFrame com o Index:Data da Coleta, Colunas: Bandeira, Valores: Etanol para fazer uma correlação.
etanol_n_novodf = etanol_n.pivot(index='Data da Coleta', columns='Bandeira', values='ETANOL')
# Fazendo a correlação do preço do etanol de bandeira para outra bandeira.
# Filtrando os dados faltantes com o Fillna()
etanol_correlacao_n = etanol_n_novodf.fillna(0).corr()

# Criando um heatmap para a correlação usando a função plt
plt.figure(figsize=(12, 10))
sns.heatmap(etanol_correlacao_n, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
plt.title('Correlação entre Bandeiras de Etanol na Região Norte')
plt.show()

#### ***Gasolina***

In [ ]:
# Utilizamos essa função para dropar a coluna Etanol, criar um novo dataframe e renomear o nome de
# index que estava como Produto, removendo o nome e deixando o index sem nome e reindexando.
gasolina_n = combustivel_bandeira.drop('ETANOL', axis=1).copy().rename_axis(None, axis=1).reset_index()
# Selecionando a Região da gasolina
gasolina_n = gasolina_n[gasolina['Regiao - Sigla'] == 'N']
# Criando um novo DataFrame com o Index:Data da Coleta, Colunas: Bandeira, Valores: Gasolina para fazer uma correlação.
gasolina_n_novodf = gasolina_n.pivot(index='Data da Coleta', columns='Bandeira', values='GASOLINA')
# Fazendo a correlação do preço do etanol de bandeira para outra bandeira.
# Filtrando os dados faltantes com o Fillna()
gasolina_correlacao_n = gasolina_n_novodf.fillna(0).corr()
# Criando um heatmap para a correlação usando a função plt
plt.figure(figsize=(12, 10))
sns.heatmap(gasolina_correlacao_ne, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
plt.title('Correlação entre Bandeiras da Gasolina na Região Norte')
plt.show()

### ***Região Centro-Oeste***



#### ***Etanol***

In [ ]:
# Utilizamos essa função para dropar a coluna Gasolina, criar um novo dataframe e renomear o nome de
# index que estava como Produto, removendo o nome e deixando o index sem nome e reindexando.
etanol = combustivel_bandeira.drop('GASOLINA', axis=1).copy().rename_axis(None, axis=1).reset_index()
# Selecionando a Região do Etanol
etanol_co = etanol[etanol['Regiao - Sigla'] == 'CO']
# Criando um novo DataFrame com o Index:Data da Coleta, Colunas: Bandeira, Valores: Etanol para fazer uma correlação.
etanol_co_novodf = etanol_co.pivot(index='Data da Coleta', columns='Bandeira', values='ETANOL')
# Fazendo a correlação do preço do etanol de bandeira para outra bandeira.
# Filtrando os dados faltantes com o Fillna()
etanol_correlacao_co = etanol_co_novodf.fillna(0).corr()
# Criando um heatmap para a correlação usando a função plt
plt.figure(figsize=(12, 10))
sns.heatmap(etanol_correlacao_co, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
plt.title('Correlação entre Bandeiras de Etanol na Região Centro-Oeste')
plt.show()

#### ***Gasolina***

In [ ]:
# Utilizamos essa função para dropar a coluna Etanol, criar um novo dataframe e renomear o nome de
# index que estava como Produto, removendo o nome e deixando o index sem nome e reindexando.
gasolina_n = combustivel_bandeira.drop('ETANOL', axis=1).copy().rename_axis(None, axis=1).reset_index()
# Selecionando a Região da Gasolina
gasolina_co = gasolina[gasolina['Regiao - Sigla'] == 'CO']
# Criando um novo DataFrame com o Index:Data da Coleta, Colunas: Bandeira, Valores: Gasolina para fazer uma correlação.
gasolina_co_novodf = gasolina_co.pivot(index='Data da Coleta', columns='Bandeira', values='GASOLINA')
# Fazendo a correlação do preço da Gasolina de bandeira para outra bandeira.
# Filtrando os dados faltantes com o Fillna()
gasolina_correlacao_co = gasolina_co_novodf.fillna(0).corr()
# Criando um heatmap para a correlação usando a função plt
plt.figure(figsize=(12, 10))
sns.heatmap(gasolina_correlacao_co, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
plt.title('Correlação entre Bandeiras da Gasolina na Região Centro-Oeste')
plt.show()

### ***Região Sudeste***

#### ***Etanol***

In [ ]:
# Utilizamos essa função para dropar a coluna Gasolina, criar um novo dataframe e renomear o nome de
# index que estava como Produto, removendo o nome e deixando o index sem nome e reindexando.
etanol = combustivel_bandeira.drop('GASOLINA', axis=1).copy().rename_axis(None, axis=1).reset_index()
# Selecionando a Região do Etanol
etanol_se = etanol[etanol['Regiao - Sigla'] == 'SE']
# Criando um novo DataFrame com o Index:Data da Coleta, Colunas: Bandeira, Valores: Etanol para fazer uma correlação.
etanol_se_novodf = etanol_se.pivot(index='Data da Coleta', columns='Bandeira', values='ETANOL')
# Fazendo a correlação do preço do etanol de bandeira para outra bandeira.
# Filtrando os dados faltantes com o Fillna()
etanol_correlacao_se = etanol_se_novodf.fillna(0).corr()
# Criando um heatmap para a correlação usando a função plt
plt.figure(figsize=(12, 10))
sns.heatmap(etanol_correlacao_se, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
plt.title('Correlação entre Bandeiras de Etanol na Região Sudeste')
plt.show()

#### ***Gasolina***

In [ ]:
# Utilizamos essa função para dropar a coluna Etanol, criar um novo dataframe e renomear o nome de
# index que estava como Produto, removendo o nome e deixando o index sem nome e reindexando.
gasolina_n = combustivel_bandeira.drop('ETANOL', axis=1).copy().rename_axis(None, axis=1).reset_index()
# Selecionando a Região da Gasolina
gasolina_se = gasolina[gasolina['Regiao - Sigla'] == 'SE']
# Criando um novo DataFrame com o Index:Data da Coleta, Colunas: Bandeira, Valores: Gasolina para fazer uma correlação.
gasolina_se_novodf = gasolina_se.pivot(index='Data da Coleta', columns='Bandeira', values='GASOLINA')
# Fazendo a correlação do preço da Gasolina de bandeira para outra bandeira.
# Filtrando os dados faltantes com o Fillna()
gasolina_correlacao_se = gasolina_se_novodf.fillna(0).corr()
# Criando um heatmap para a correlação usando a função plt
plt.figure(figsize=(12, 10))
sns.heatmap(gasolina_correlacao_se, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
plt.title('Correlação entre Bandeiras da Gasolina na Região Sudeste')
plt.show()

### ***Região Sul***

#### ***Etanol***

In [ ]:
# Utilizamos essa função para dropar a coluna Gasolina, criar um novo dataframe e renomear o nome de
# index que estava como Produto, removendo o nome e deixando o index sem nome e reindexando.
etanol = combustivel_bandeira.drop('GASOLINA', axis=1).copy().rename_axis(None, axis=1).reset_index()
# Selecionando a Região do Etanol
etanol_s = etanol[etanol['Regiao - Sigla'] == 'S']
# Criando um novo DataFrame com o Index:Data da Coleta, Colunas: Bandeira, Valores: Etanol para fazer uma correlação.
etanol_s_novodf = etanol_s.pivot(index='Data da Coleta', columns='Bandeira', values='ETANOL')
# Fazendo a correlação do preço do etanol de bandeira para outra bandeira.
# Removendo linhas e colunas em branco
etanol_correlacao_s = etanol_correlacao_s.replace(0, pd.NA).dropna(axis=0, how='all').dropna(axis=1, how='all')
# Criando um heatmap para a correlação usando a função plt
plt.figure(figsize=(12, 10))
sns.heatmap(etanol_correlacao_s, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
plt.title('Correlação entre Bandeiras de Etanol na Região Sul')
plt.show()

#### ***Gasolina***

In [ ]:
# Utilizamos essa função para dropar a coluna Etanol, criar um novo dataframe e renomear o nome de
# index que estava como Produto, removendo o nome e deixando o index sem nome e reindexando.
gasolina_n = combustivel_bandeira.drop('ETANOL', axis=1).copy().rename_axis(None, axis=1).reset_index()
# Selecionando a Região da Gasolina
gasolina_s = gasolina[gasolina['Regiao - Sigla'] == 'S']
# Criando um novo DataFrame com o Index:Data da Coleta, Colunas: Bandeira, Valores: Gasolina para fazer uma correlação.
gasolina_s_novodf = gasolina_s.pivot(index='Data da Coleta', columns='Bandeira', values='GASOLINA')
# Fazendo a correlação do preço da Gasolina de bandeira para outra bandeira.
# Filtrando os dados faltantes com o Fillna()
gasolina_correlacao_s = gasolina_s_novodf.fillna(0).corr()
# Criando um heatmap para a correlação usando a função plt
plt.figure(figsize=(12, 10))
sns.heatmap(gasolina_correlacao_s, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
plt.title('Correlação entre Bandeiras da Gasolina na Região Sul')
plt.show()

##***Perguntas Extra***

###*11. Qual a distribuição de bandeiras por região do país?*

In [ ]:
# Contando o número de bandeiras em cada região
distribuicao_bandeiras = dfproc.groupby(['Regiao - Sigla', 'Bandeira']).size().reset_index(name='Contagem')

# Agrupando os dados por região e bandeira e contando o número de ocorrências
distribuicao_bandeiras = distribuicao_bandeiras.groupby(['Regiao - Sigla', 'Bandeira'])['Contagem'].sum().unstack(fill_value=0)

# Imprimindo a distribuição de bandeiras por região formatando-a
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(distribuicao_bandeiras)



###*12. Existe alguma relação entre a data do mês em que determinadas bandeiras são mais lucrativas?*

In [ ]:
# Extraindo o mês da data da coleta
dfproc['Mes'] = dfproc['Data da Coleta'].dt.month

# Calculando a soma do valor de venda para cada bandeira em cada mês
lucro_por_mes = dfproc.groupby(['Mes', 'Bandeira'])['Valor de Venda'].sum().reset_index()

# Encontrando o mês de maior lucro para cada bandeira
mes_maior_lucro = lucro_por_mes.loc[lucro_por_mes.groupby('Bandeira')['Valor de Venda'].idxmax()]

# Removendo o índice numérico
mes_maior_lucro.set_index('Bandeira', inplace=True)

# Formatando a saída
mes_maior_lucro_styled = mes_maior_lucro.style.format({'Valor de Venda': 'R$ {:,.2f}'}).background_gradient(cmap='Blues')

# Imprimindo o mês de maior lucro para cada bandeira
display(mes_maior_lucro_styled)




###*13. Existe alguma bandeira exclusiva de alguns estados?*

In [ ]:
# Para cada estado, percorrer lista de bandeira e arquivar e em dataframe


In [ ]:
# Agrupando os dados por estado e bandeira
bandeiras_por_estado = dfproc.groupby(['Estado - Sigla', 'Bandeira']).size().reset_index(name='Contagem')

# Encontrando as bandeiras que aparecem apenas em um estado
bandeiras_exclusivas = bandeiras_por_estado['Bandeira'].value_counts()
bandeiras_exclusivas = bandeiras_exclusivas[bandeiras_exclusivas == 1]

# Imprimindo as bandeiras exclusivas de alguns estados
print("Bandeiras exclusivas de alguns estados:")
print(bandeiras_exclusivas)






In [ ]:
# Agrupando os dados por estado e bandeira
bandeiras_por_estado = dfproc.groupby(['Estado - Sigla', 'Bandeira']).size().reset_index(name='Contagem')

# Encontrando as bandeiras que aparecem apenas em um estado
bandeiras_exclusivas = bandeiras_por_estado['Bandeira'].value_counts()
bandeiras_exclusivas = bandeiras_exclusivas[bandeiras_exclusivas == 1]

# Obtendo os estados correspondentes às bandeiras exclusivas
estados_bandeiras_exclusivas = bandeiras_por_estado[bandeiras_por_estado['Bandeira'].isin(bandeiras_exclusivas.index)]

# Removendo a coluna 'Contagem'
estados_bandeiras_exclusivas = estados_bandeiras_exclusivas.drop(columns='Contagem')

# Formatando a saída
estados_bandeiras_exclusivas_styled = estados_bandeiras_exclusivas.style.hide_index().set_properties(**{'background-color': 'lightblue', 'color': 'black'})

# Imprimindo as bandeiras exclusivas de alguns estados
display(estados_bandeiras_exclusivas_styled)

